# 第7章　音声信号処理の基礎（確認問題の解答）
コードのセルを，本文ノートブック chap7.ipynb にコピペして実行してください。


## 7.1 音声認識の基礎 
(1) 上記の(b)では，高ケフレンシー部だけに着目するため，低ケフレンシー部の値を 0 とした。これは，横軸が周波数の場合には高域通過フィルタをかけたことに相当し，リフタ (lifter)と呼ばれる（filter のスペルを一部反転させた造語）。さて，逆に低ケフレンシー部だけ値を残して，FFTをかけると「ソース － フィルタ」のうちフィルタの周波数特性が得られるはずである。実際に行ってみること。

【答】本文のノートブックにおける 7.1節 (b) （ステップ２）まで実行してください。そして，（ステップ３）の部分に＋アイコンで新しいセルを2つ設け，以下の2つのセルをコピーして実行してください。

In [ ]:
cep_temp = np.copy(cep)  # 元の配列 cep の内容を保存するため，cep_temp にコピーして 
cep_temp[50:-50] = 0     # 50番以上の成分を 0 にします。ケフレンシー軸の右半分（対称な部分）に同じ処理をします。
plot_wave([], np.abs(cep_temp), xtitle = 'Quefrency (point)')  # 横軸は右半分も含めて表示します。

ケプストラムを求めたのと逆の手順で，FFT して，指数関数で戻します。結果を赤線で，元々のスペクトルを青線でしまします。

In [ ]:
upper_freq = 256
formants = np.exp(np.fft.fft(cep_temp))
draw_FFT_spectrum(sp[0:upper_freq], fs = 8000, phase_spectrum = False, stem = False, level = True, draw_range = 90, hold = True)
draw_FFT_spectrum(formants[0:upper_freq], fs = 8000, phase_spectrum = False, stem = False, level = True, draw_range = 90, color='r')

内側のギザギザが消えて，フォルマントが分かりやすくなりましたね。 

[演習] ここでは 50 次以上のケプストラム係数を 0 にした。これを 10, 20,... と変えて実行してみなさい。

(2) 上記の(b)では，ピッチ周波数の抽出をケプストラムを利用して行った。他にも多くの方法が知られているので，調べること。

【答】ケプストラムと同様に「スペクトル」に基づく処理として，ピリオドヒストグラムを用いる手法がある。他に，「波形」に基づく処理として，零交叉計数法・並列処理法・データ減少法がある。さらには波形の「相関」に基づく処理として，自己相関法・変形相関法・SIFTアルゴリズム・AMDFなどがある。

(3) 自身の音声/a/, /i/, /u/, /e/, /o/ についてケプストラム係数を算出してテンプレートし，(c) の音声認識に挑戦すること。

【答】省略。

(4) 音声認識の性能を上げるために，実際のシステムではどのような工夫がなされているか調べること。

【答】少なくとも，以下のキーワードについて調べてください。
+ プリエンファシス
+ MFCC (mel-frequency cepstral coefficient,　メル周波数ケプストラム係数)
+ CMS (cepstrum mean subtraction, ケプストラム平均除去)
+ 混合分布，代表的なものとしてGMM (Gaussian mixture odel, 混合ガウスモデル)
+ 話者適応，代表的なものとしてMLLR (maximum likelihood linear regression, 最尤線形回帰法)
+ 以上は，認識に用いる「音響特徴量」に関わる項目ですが，さらに認識アルゴリズムに関して「音響モデルと言語モデル」「統計的パターン認識（識別モデルと生成モデル）」「HMM (hidden Markov model)」「深層学習」など...

## 7.2 線形予測分析による音声の分析 

(1) 7.1節の確認課題 (1) では，リフタにより低ケフレンシー部だけ値を残してFFTをかけることで，「ソース － フィルタ」のうちフィルタの周波数特性を観測している。その結果と，線形予測分析によって得た結果を比較しなさい。

【答】どちらもスペクトル内の外形を抽出するものであるが，ケプストラム係数および線形予測係数の次数をどこまで用いるかで形が変わるので，統一的な評価は困難である。ただ，一般的には，線形予測分析によって得た結果は「スペクトルのピークを重視した形」になるといえる（興味のある方は「全極モデル」といったキーワードでさらに学習してください）。

(2) LPC係数を保存する配列 LPCcoef（サイズは5(母音)×64(係数)）を用意しなさい。そして，LPCの次数 p = 32 の場合について，各母音 /a/, /i/, /u/, /e/, /o/ のそれぞれの線形予測係数を保存しなさい。

【答】例えば以下のようなプログラムで実行可能です。本文のインプット 6.15の下 に「＋コード」で新しいセルを設けて，以下をコピーして実行してください。

In [ ]:
#6.2節 (b) のプログラムにおいて，target という変数を for ループで回すことで，値を記録できます。
fs, wave_data = scipy.io.wavfile.read ('sample/myAIUEO.wav') # WAVファイルから，標本化周波数と音声データをそれぞれ fs, wave_data に読み込みます

nfft = 512       # FFTの点数を512とします。
start = np.array([15000, 21000, 33000, 45000, 60000]) # 各母音の開始ポイントをまとめて宣言します。

p = 32   # LPCの次数
LPCcoef = np.zeros((5, p))  # LPC係数を保存する配列の準備

for target in range(5):   # 対象を順番に回しながら...
    voice_interval = (start[target], start[target] + nfft) # 対象音の切り出し区間を設定します。

    voice_data = wave_data[ voice_interval[0] : voice_interval[1] ] # 対象となるデータを取り出します。
    voice_data = hanning( len(voice_data) ) * voice_data # FFT する前にハニング窓をかけておきます。

    # LPC係数を求めます。
    r = autocorr(voice_data, p + 1)
    a, e  = LevinsonDurbin(r, p)
    LPCcoef[target] = a[1:p+1]  # \alpha_0 は 1 なので記録を省略

(3) この節では，数式の導出を大幅に省略している。ぜひ，信号処理の教科書を読んで，z変換に関わる部分など自習されたい。

【答】省略。

## 7.3 線形予測分析による音声の合成 

(1) ここでは，ピッチ周波数 f0 = 125 [Hz] として合成を行った。これを，例えば 250 [Hz] とすると，女声に聞こえるか試しなさい。

【答】インプット 6.16において f0 = 250.0 と変更して実行してください。かなり微妙ですね。これは，女声のそれらしさは f0 だけでなく，フォルマント周波数にも依存していることの証です。

(2) 自身の声について線形予測係数を求めて，上記の合成を行いなさい。

【答】省略。もし読者が女性であれば， f0 = 250.0 などとする方が自然な音声になるでしょう。

(3) 線形予測の次数 lpcOrder = 32 を 4, 8, 16, 64 と変えて実行し，音質の違いを検討しなさい。

【答】省略。

(4) ソースを白色雑音にすることで，無声音（ささやき声）となるか確認しなさい。
【答】6.3節(a) のプログラム中でパルス列 s を合成している部分がある。その部分の下に，例えば以下のような行を加えれば，ソースを雑音に変更できる。

In [ ]:
s = np.random.randn(len(t)) # 無声音の場合

(5) ソースをパルス列としたとき，パルス間隔に揺らぎを与えると，どのような変化が生ずるか確認しなさい。

【答】7.3節 (a) のインプット7.16 中にパルス列を作っている部分がある。その中に，以下のように「#★コメント文」の行を2行挿入すればよい。どの程度の揺らぎを与えるかを，正規分布の標準偏差（とりあえず 5）で調整できる。揺らぎが全くない場合に比べると，自然性が若干向上すると感じられるであろう。

In [ ]:
pulse_points = arange(0, len(t), pulse_interval) # パルスの立つ離散時刻を定めます。
for i in arange(len(pulse_points)):                    #★ パルスの立つ離散時刻の各点について，
    pulse_points[i] += int( (np.random.normal(0, 5)) ) #★ ゆらぎを与えます。
s[pulse_points] = 1               # その離散時刻の値を 1 にします。